构建两个神经网络，一个是构建两层的神经网络，一个是构建多层的神经网络，多层神经网络的层数可以自己定义。本次的教程的难度有所提升，但是我会力求深入简出。在这里，我们简单的讲一下难点，本文会提到`LINEAR-> ACTIVATION`转发函数，比如我有一个多层的神经网络，结构是`输入层->隐藏层->隐藏层->···->隐藏层->输出层`，在每一层中，我会首先计算`Z = np.dot(W,A) + b`，这叫做`linear_forward`，然后再计算`A = relu(Z)` 或者 `A = sigmoid(Z)`，这叫做`linear_activation_forward`，合并起来就是这一层的计算方法，所以每一层的计算都有两个步骤，先是计算Z，再计算A

<img src="./picture/img_4.png" width="70%">

步骤：

1. 初始化网络参数

2. 前向传播

    1. 计算一层的中线性求和的部分

    2. 计算激活函数的部分（ReLU使用L-1次，Sigmod使用1次）

    3. 结合线性求和与激活函数

3. 计算误差

4. 反向传播

    1. 线性部分的反向传播公式

    2. 激活函数部分的反向传播公式

    3. 结合线性部分与激活函数的反向传播公式

5. 更新参数

#### 准备软件包

In [5]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

np.random.seed(1)

#### 数据加载函数

In [2]:
def load_dataset():

    # 读取训练集
    train_dataset = h5py.File('datasets/train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset['train_set_x'][:]) # 加 [:] 是把数据从 h5py.Dataset 转换为 numpy.ndarray;外层再用 np.array() 是为了确保类型一致（有时是冗余的，但保险）。
    train_set_y_orig = np.array(train_dataset['train_set_y'][:])

    # 读取测试集
    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset['test_set_x'][:])
    test_set_y_orig = np.array(test_dataset['test_set_y'][:])

    # 获取类别标签
    classes = np.array(test_dataset['list_classes'][:])

    # 标签 reshape 处理
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0])) # 原始标签形状是 (m,)，现在 reshape 成 (1, m)，很多神经网络实现中要求标签是形如 (1, m) 的二维数组
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    # 返回所有数据
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

#### 常见激活函数

In [3]:
def sigmoid(Z):
    """
    实现 Sigmoid 激活函数（用 NumPy 实现）

    Arguments:
    Z -- 任意形状的 numpy 数组

    Returns:
    A -- sigmoid(Z) 的输出，形状与 Z 相同
    cache -- 返回 Z，用于反向传播时使用
    """
    A = 1/(1+np.exp(-Z))
    cache = Z

    return A, cache

def sigmoid_backward(dA, cache):
    """
    实现单个 Sigmoid 单元的反向传播

    Arguments:
    dA -- 激活后的梯度，任意形状

    cache -- 前向传播时保存的 Z，用于高效地计算反向传播

    Returns:
    dZ -- 成本对 Z 的梯度
    """
    Z = cache

    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)

    assert (dZ.shape == Z.shape)

    return dZ

def relu(Z):
    """
    实现 ReLU 激活函数

    Arguments:
    Z -- 线性层的输出，任意形状

    Returns:
    A -- 激活后的输出，形状与 Z 相同

    cache -- 一个包含 Z 的变量，用于高效地进行反向传播计算
    """
    A = np.maximum(0,Z)

    assert(A.shape == Z.shape)

    cache = Z
    return A, cache

def relu_backward(dA, cache):
    """
    实现单个 ReLU 单元的反向传播

    Arguments:
    dA -- 激活后的梯度，任意形状

    cache -- 前向传播时保存的 Z，用于高效地计算反向传播

    Returns:
    dZ -- 成本对 Z 的梯度
    """

    Z = cache
    dZ = np.array(dA, copy=True) 

    dZ[Z <= 0] = 0

    assert (dZ.shape == Z.shape)

    return dZ


- testCases：提供了一些测试示例来评估函数的正确性，参见下载的资料或者在底部查看它的代码。

每个函数都用来模拟某个阶段的输入和输出，以便你在实现具体算法时可以用这些数据来验证自己代码是否正确。

In [14]:
# 构造用于测试线性前向传播的输入：输入激活值 A、权重矩阵 W 和偏置项 b。
def linear_forward_test_case():
    np.random.seed(1)
    A = np.random.randn(3,2) # 随机生成一个 3x2 的矩阵 A
    W = np.random.randn(1,3) # 随机生成一个 1x3 的矩阵 W
    b = np.random.randn(1,1) # 随机生成一个 1x1 的矩阵 b

    return A,W,b

# 构造用于测试“线性计算+激活函数”的输入参数
def linear_activation_forward_test_case():
    np.random.seed(2)
    A_prev = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)

    return A_prev, W, b

# 生成一个 2 层神经网络的参数和输入 X，用于测试整个 L 层前向传播函数。
def L_model_forward_test_case():
    np.random.seed(1)
    X = np.random.randn(4,2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return X, parameters

# 生成用于测试损失函数（交叉熵）计算的预测值 aL 和真实标签 Y。
def compute_cost_test_case():
    Y = np.array([[1, 1, 1]]) # 三个样本的真实标签，全是1
    aL = np.array([[.8, .9, 0.4]]) # 三个样本的预测值，也叫“激活值”，是神经网络的输出

    return Y, aL

# 用于测试线性部分的反向传播函数。
def linear_backward_test_case():
    np.random.seed(1)
    dZ = np.random.randn(1,2)
    A = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)

    linear_cache = (A, W, b)

    return dZ, linear_cache

# 用于测试带激活函数的反向传播。
def linear_activation_backward_test_case():
    np.random.seed(2)
    dA = np.random.randn(1,2) # 当前层的激活梯度 dA，1x2
    A = np.random.randn(3,2) # 前一层的激活值，3x2
    W = np.random.randn(1,3) # 权重矩阵，1x3
    b = np.random.randn(1,1) # 偏置项，1x1
    Z = np.dot(W.T, A) + b # 当前层的 Z（线性输出）

    activation_cache = (Z) # 激活函数缓存

    linear_cache = (A, W, b) # 前向传播缓存
    linear_activation_cache = (linear_cache, activation_cache) # 合并缓存

    return dA, linear_activation_cache

# 生成完整神经网络的反向传播输入，包括前一层和后一层的 cache。
def L_model_backward_test_case():
    np.random.seed(3)
    AL = np.random.randn(1,2) # 最后一层的输出预测值
    Y = np.array([[1, 0]]) # 真实标签，只有两个样本

    A1 = np.random.randn(4,2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    Z1 = np.random.randn(3,2)
    linear_cache_activation_1 = ((A1, W1, b1), Z1) # 第一层缓存

    A2 = np.random.randn(3, 2)
    W2 = np.random.randn(1, 3)
    b2 = np.random.randn(1, 1)
    Z2 = np.random.randn(1, 2)
    linear_cache_activation_2 = ((A2, W2, b2), Z2) # 第二层缓存

    caches = (linear_cache_activation_1, linear_cache_activation_2)

    return AL, Y, caches

# 生成测试用的参数和梯度，用于测试参数更新函数。
def update_parameters_test_case():
    np.random.seed(2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    np.random.seed(3)
    dW1 = np.random.randn(3,4)
    db1 = np.random.randn(3,1)
    dW2 = np.random.randn(1,3)
    db2 = np.random.randn(1,1)

    grads = {
        "dW1": dW1,
        "db1": db1,
        "dW2": dW2,
        "db2": db2
    }

    return parameters, grads

## 初始化参数
### （1）对于一个两层的神经网络结构而言，模型结构是线性->ReLU->线性->sigmod函数。

In [7]:
def initialize_parameters(n_x, n_h, n_y):
    """
    此函数是为了初始化两层网络参数而使用的函数。
    参数：
        n_x - 输入层节点数量
        n_h - 隐藏层节点数量
        n_y - 输出层节点数量

    返回：
        parameters - 包含你的参数的python字典：
            W1 - 权重矩阵,维度为（n_h，n_x）
            b1 - 偏向量，维度为（n_h，1）
            W2 - 权重矩阵，维度为（n_y，n_h）
            b2 - 偏向量，维度为（n_y，1）

    """
    W1 = np.random.randn(n_h, n_x) * 0.01
    W2 = np.random.randn(n_y, n_h) * 0.01
    b1 = np.random.randn(n_h, 1)
    b2 = np.random.randn(n_y, 1)

    parameters = {
        "W1": W1,
        "b1": b1,
        "W2": W2,
        "b2": b2
    }

    return parameters

# 测试
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))


W1 = [[-0.02060141 -0.00322417 -0.00384054]
 [ 0.01133769 -0.01099891 -0.00172428]]
b1 = [[ 0.58281521]
 [-1.10061918]]
W2 = [[-0.00877858  0.00042214]]
b2 = [[1.14472371]]


### （2）初始化多层网络参数

In [13]:
def initialize_parameters_deep(layer_dims):
    """
    此函数是为了初始化多层网络参数而使用的函数。
    参数：
        layers_dims - 包含我们网络中每个图层的节点数量的列表

    返回：
        parameters - 包含参数“W1”，“b1”，...，“WL”，“bL”的字典：
                     W1 - 权重矩阵，维度为（layers_dims [1]，layers_dims [1-1]）
                     bl - 偏向量，维度为（layers_dims [1]，1）
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L): # 输入层定义为0层，故从1开始
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1]) # Xavier 初始化, 缓解梯度爆炸和梯度消失
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))

        # 断言
        assert(parameters["W" + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters["b" + str(l)].shape == (layer_dims[l], 1))

    return parameters

# 测试
layer_dims = [5,4,3,2]
print(len(layer_dims))
parameters = initialize_parameters_deep(layer_dims)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))
print("W3 = " + str(parameters["W3"]))
print("b3 = " + str(parameters["b3"]))

4
W1 = [[ 0.79989897  0.19521314  0.04315498 -0.83337927 -0.12405178]
 [-0.15865304 -0.03700312 -0.28040323 -0.01959608 -0.21341839]
 [-0.58757818  0.39561516  0.39413741  0.76454432  0.02237573]
 [-0.18097724 -0.24389238 -0.69160568  0.43932807 -0.49241241]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.59252326 -0.10282495  0.74307418  0.11835813]
 [-0.51189257 -0.3564966   0.31262248 -0.08025668]
 [-0.38441818 -0.11501536  0.37252813  0.98805539]]
b2 = [[0.]
 [0.]
 [0.]]
W3 = [[-0.71829494 -0.36166197 -0.46405457]
 [-1.39665832 -0.53335157 -0.59113495]]
b3 = [[0.]
 [0.]]


- 我们分别构建了两层和多层神经网络的初始化参数的函数，现在我们开始构建前向传播函数。

## 前向传播函数
前向传播有以下三个步骤:

1. LINEAR
2. LINEAR - >ACTIVATION，其中激活函数将会使用ReLU或Sigmoid。
3. [LINEAR - > RELU] ×（L-1） - > LINEAR - > SIGMOID（整个模型）

线性正向传播模块（向量化所有示例）使用公式(3)进行计算：

<img src="./picture/img_5.png" width="30%">


### 线性部分【LINEAR】
前向传播中，线性部分计算如下：

In [17]:
def linear_forword(A, W, b):
    """
    实现前向传播的线性部分。

    参数：
        A - 来自上一层（或输入数据）的激活，维度为(上一层的节点数量，示例的数量）
        W - 权重矩阵，numpy数组，维度为（当前图层的节点数量，前一图层的节点数量）
        b - 偏向量，numpy向量，维度为（当前图层节点数量，1）

    返回：
         Z - 激活功能的输入，也称为预激活参数
         cache - 一个包含“A”，“W”和“b”的字典，存储这些变量以有效地计算后向传递
    """
    Z = np.dot(W, A) + b
    assert (Z.shape == (W.shape[0], A.shape[1]))

    cache = (A, W, b)

    return Z, cache

# 测试
A, W, b = linear_forward_test_case()
Z, linear_cache = linear_forword(A, W, b)
print("Z = " + str(Z))
print("linear_cache = " + str(linear_cache))


Z = [[ 3.26295337 -1.23429987]]
linear_cache = (array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ]]), array([[ 1.74481176, -0.7612069 ,  0.3190391 ]]), array([[-0.24937038]]))


前向传播的单层计算完成了一半啦！我们来开始构建后半部分

### 线性激活部分【LINEAR - >ACTIVATION】
为了更方便，我们将把两个功能（线性和激活）分组为一个功能（LINEAR-> ACTIVATION）。 因此，我们将实现一个执行LINEAR前进步骤，然后执行ACTIVATION前进步骤的功能。我们来看看这激活函数的数学实现吧~

<img src="./picture/img_6.png" width="50%">

In [18]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    实现LINEAR-> ACTIVATION 这一层的前向传播

    参数：
        A_prev - 来自上一层（或输入层）的激活，维度为(上一层的节点数量，示例数）
        W - 权重矩阵，numpy数组，维度为（当前层的节点数量，前一层的大小）
        b - 偏向量，numpy阵列，维度为（当前层的节点数量，1）
        activation - 选择在此层中使用的激活函数名，字符串类型，【"sigmoid" | "relu"】

    返回：
        A - 激活函数的输出，也称为激活后的值
        cache - 一个包含“linear_cache”和“activation_cache”的字典，我们需要存储它以有效地计算后向传递
    """
    if activation == "sigmoid":
        Z, linear_cache = linear_forword(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forword(A_prev, W, b)
        A, activation_cache = relu(Z)

    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

# 测试
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]
